Case study 1: Single-Category Classification using N24News Dataset
-------------------------------------------------------------

This demo shows how to classify a single news article into one of 24 category
using the N24News dataset. Each article in the dataset includes both textual
and visual information.

Source: https://aclanthology.org/2022.lrec-1.729/


Each article contains the following fields:
- 'section': Ground truth label (one of 24 category)
- 'headline': Title of the article
- 'abstract': Short summary of the article
- 'article': Full text content
- 'article_url': Link to the original article
- 'image': Encoded image or metadata (optional)
- 'caption': Image caption
- 'image_id': Unique image identifier
- 'image_path': Path to the associated image (e.g., 'N24News/imgs_200_sample1/12345.jpg')
- 'article_id': Unique article identifier

Image file: Multimodal_image

Example category (See prompt_D1 for the complete category):
------------------------
1. Health
2. Science
3. Television
...
24. Global Business

Reference:
----------
Wang, Z., Shan, X., Zhang, X., & Yang, J. (2022).
N24News: A New Dataset for Multimodal News Classification.
In *Proceedings of the Thirteenth Language Resources and Evaluation Conference* (pp. 6768–6775). LREC.


In [2]:
import os
import pandas as pd

In [3]:
from labelgenius import (
    classification_CLIP_0_shot,
    classification_CLIP_finetuned,
    finetune_CLIP,
    auto_verification,
    classification_GPT,
    generate_GPT_finetune_jsonl,
    finetune_GPT,
    price_estimation,
)

Using device: mps


# Test with GPT

In [4]:
import os
import gc
import io
import pandas as pd
from contextlib import redirect_stdout, redirect_stderr
from sklearn.metrics import accuracy_score

# --- Settings ---
# Define the descriptive prompt 
prompt_GPT = ["""You are given a short news article and an image. Based on its content, return the most appropriate category label from the list below. Respond with only a number from 1 to 24, corresponding to the best-matching category:

1. Health – medical news, public health, fitness, mental health, wellness
2. Science – scientific discoveries, research studies, space, innovations
3. Television – TV shows, reviews, industry news, streaming
4. Travel – tourism, destinations, travel guides, airlines, vacation trends
5. Movies – film news, reviews, box office, upcoming releases
6. Dance – ballet, contemporary styles, street dance, performances
7. Real Estate – housing trends, property sales, architecture, urban planning
8. Economy – macroeconomics, inflation, markets, GDP, financial policies
9. Sports – competitions, athletes, Demo_results, professional sports
10. Theater – plays, Broadway, live performances, stage reviews
11. Opinion – editorials, commentary, analysis
12. Music – albums, artists, concerts, festivals, music trends
13. Books – literature, bestsellers, author interviews, book reviews
14. Art & Design – fine arts, museums, exhibitions, visual/design trends
15. Style – fashion trends, beauty, personal style, aesthetics
16. Media – journalism, publishing, digital media, mass communication
17. Food – restaurants, recipes, cooking, culinary culture
18. Well – lifestyle, self-care, mental well-being, personal development
19. Fashion – clothing, designers, fashion weeks, industry insights
20. Technology – AI, gadgets, software, cybersecurity, tech innovations
21. Your Money – personal finance, investing, budgeting, financial planning
22. Education – schools, universities, learning methods, education policies
23. Automobiles – car news, EVs, reviews, industry trends
24. Global Business – international trade, corporations, mergers, global markets

Return the categorty label only (e.g., 5), no extra words."""]



# Define the list of 24 category labels
category = [
    "1", "2", "3", "4", "5", "6",
    "7", "8", "9", "10", "11", "12",
    "13", "14", "15", "16", "17", "18",
    "19", "20", "21", "22", "23", "24"
]
api_key = ""


## Estimate Price

In [5]:
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI()

In [6]:
response_41 = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "system",
      "content": [
        {
          "type": "input_text",
          "text": "\"\"\"You are given a short news article. Based on its content, return the most appropriate category label from the list below. Respond with only a number from 1 to 24, corresponding to the best-matching category:\n\n1. Health – medical news, public health, fitness, mental health, wellness\n2. Science – scientific discoveries, research studies, space, innovations\n3. Television – TV shows, reviews, industry news, streaming\n4. Travel – tourism, destinations, travel guides, airlines, vacation trends\n5. Movies – film news, reviews, box office, upcoming releases\n6. Dance – ballet, contemporary styles, street dance, performances\n7. Real Estate – housing trends, property sales, architecture, urban planning\n8. Economy – macroeconomics, inflation, markets, GDP, financial policies\n9. Sports – competitions, athletes, Demo_results, professional sports\n10. Theater – plays, Broadway, live performances, stage reviews\n11. Opinion – editorials, commentary, analysis\n12. Music – albums, artists, concerts, festivals, music trends\n13. Books – literature, bestsellers, author interviews, book reviews\n14. Art & Design – fine arts, museums, exhibitions, visual/design trends\n15. Style – fashion trends, beauty, personal style, aesthetics\n16. Media – journalism, publishing, digital media, mass communication\n17. Food – restaurants, recipes, cooking, culinary culture\n18. Well – lifestyle, self-care, mental well-being, personal development\n19. Fashion – clothing, designers, fashion weeks, industry insights\n20. Technology – AI, gadgets, software, cybersecurity, tech innovations\n21. Your Money – personal finance, investing, budgeting, financial planning\n22. Education – schools, universities, learning methods, education policies\n23. Automobiles – car news, EVs, reviews, industry trends\n24. Global Business – international trade, corporations, mergers, global markets\n\nReturn the categorty label only (e.g., 5), no extra words.\"\"\""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "36 Hours in Lake Tahoe; Straddling Nevada and California, Lake Tahoe is a winter playground for skiers and snowboarders, food enthusiasts and anyone willing to be pulled into the area's adventurous orbit.\nExpand"
        },
        {
          "type": "input_image",
          "image_url": "https://static01.nyt.com/images/2015/03/01/travel/01HOURS2/01HOURS2-watch308-v2.jpg"
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[],
  temperature=1,
  max_output_tokens=10000,
  top_p=1,
  store=True
)

In [7]:
response_41

Response(id='resp_08bf137b6c5f92b10068e6c5f47b4c819580235eea5deb6be0', created_at=1759954420.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_08bf137b6c5f92b10068e6c5f5ed488195a102b5ee27601ed1', content=[ResponseOutputText(annotations=[], text='4', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=10000, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), truncation='disabled', usage=ResponseUsage(input_tokens=726, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=2, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=728), user=None, background=

In [8]:
price_estimation (response_41,
                  num_rows = 4000,
                  input_cost_per_million = 2,
                  output_cost_per_million = 8,
                  num_votes = 3)

# https://openai.com/api/pricing/


🧮 Estimated Cost for 12,000 calls (4,000 rows × 3 votes)
• Avg prompt tokens/call:     726
• Avg completion tokens/call: 2
• Pricing ($/1M tokens): prompt=$2, completion=$8
💰 Total: $17.6160    (±10% → $15.8544–$19.3776)



17.616

In [9]:
response_5_non_reasoning = client.responses.create(
  model="gpt-5",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": "\"\"\"You are given a short news article. Based on its content, return the most appropriate category label from the list below. Respond with only a number from 1 to 24, corresponding to the best-matching category:\n\n1. Health – medical news, public health, fitness, mental health, wellness\n2. Science – scientific discoveries, research studies, space, innovations\n3. Television – TV shows, reviews, industry news, streaming\n4. Travel – tourism, destinations, travel guides, airlines, vacation trends\n5. Movies – film news, reviews, box office, upcoming releases\n6. Dance – ballet, contemporary styles, street dance, performances\n7. Real Estate – housing trends, property sales, architecture, urban planning\n8. Economy – macroeconomics, inflation, markets, GDP, financial policies\n9. Sports – competitions, athletes, Demo_results, professional sports\n10. Theater – plays, Broadway, live performances, stage reviews\n11. Opinion – editorials, commentary, analysis\n12. Music – albums, artists, concerts, festivals, music trends\n13. Books – literature, bestsellers, author interviews, book reviews\n14. Art & Design – fine arts, museums, exhibitions, visual/design trends\n15. Style – fashion trends, beauty, personal style, aesthetics\n16. Media – journalism, publishing, digital media, mass communication\n17. Food – restaurants, recipes, cooking, culinary culture\n18. Well – lifestyle, self-care, mental well-being, personal development\n19. Fashion – clothing, designers, fashion weeks, industry insights\n20. Technology – AI, gadgets, software, cybersecurity, tech innovations\n21. Your Money – personal finance, investing, budgeting, financial planning\n22. Education – schools, universities, learning methods, education policies\n23. Automobiles – car news, EVs, reviews, industry trends\n24. Global Business – international trade, corporations, mergers, global markets\n\nReturn the categorty label only (e.g., 5), no extra words.\"\"\""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "36 Hours in Lake Tahoe; Straddling Nevada and California, Lake Tahoe is a winter playground for skiers and snowboarders, food enthusiasts and anyone willing to be pulled into the area's adventurous orbit.\nExpand"
        },
        {
          "type": "input_image",
          "image_url": "https://static01.nyt.com/images/2015/03/01/travel/01HOURS2/01HOURS2-watch308-v2.jpg"
        }
      ]
    },
    {
      "type": "reasoning",
      "id": "rs_68263a6e1e5c8191856e1f5b5ee8c77a0252b23c99ac10a1",
      "summary": []
    },
    {
      "id": "msg_68263a70b4208191bb874acec2ab4e8e0252b23c99ac10a1",
      "role": "assistant",
      "content": [
        {
          "type": "output_text",
          "text": "4"
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={
    "effort": "minimal"
  },
  tools=[],
  store=True
)



In [10]:
price_estimation (response_5_non_reasoning,
                  num_rows = 4000,
                  input_cost_per_million = 1.24,
                  output_cost_per_million = 10,
                  num_votes = 3)

# https://openai.com/api/pricing/


🧮 Estimated Cost for 12,000 calls (4,000 rows × 3 votes)
• Avg prompt tokens/call:     687
• Avg completion tokens/call: 7
• Pricing ($/1M tokens): prompt=$1.24, completion=$10
💰 Total: $11.0626    (±10% → $9.9563–$12.1688)



11.06256

In [11]:
response_5_reasoning = client.responses.create(
  model="gpt-5",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": "\"\"\"You are given a short news article. Based on its content, return the most appropriate category label from the list below. Respond with only a number from 1 to 24, corresponding to the best-matching category:\n\n1. Health – medical news, public health, fitness, mental health, wellness\n2. Science – scientific discoveries, research studies, space, innovations\n3. Television – TV shows, reviews, industry news, streaming\n4. Travel – tourism, destinations, travel guides, airlines, vacation trends\n5. Movies – film news, reviews, box office, upcoming releases\n6. Dance – ballet, contemporary styles, street dance, performances\n7. Real Estate – housing trends, property sales, architecture, urban planning\n8. Economy – macroeconomics, inflation, markets, GDP, financial policies\n9. Sports – competitions, athletes, Demo_results, professional sports\n10. Theater – plays, Broadway, live performances, stage reviews\n11. Opinion – editorials, commentary, analysis\n12. Music – albums, artists, concerts, festivals, music trends\n13. Books – literature, bestsellers, author interviews, book reviews\n14. Art & Design – fine arts, museums, exhibitions, visual/design trends\n15. Style – fashion trends, beauty, personal style, aesthetics\n16. Media – journalism, publishing, digital media, mass communication\n17. Food – restaurants, recipes, cooking, culinary culture\n18. Well – lifestyle, self-care, mental well-being, personal development\n19. Fashion – clothing, designers, fashion weeks, industry insights\n20. Technology – AI, gadgets, software, cybersecurity, tech innovations\n21. Your Money – personal finance, investing, budgeting, financial planning\n22. Education – schools, universities, learning methods, education policies\n23. Automobiles – car news, EVs, reviews, industry trends\n24. Global Business – international trade, corporations, mergers, global markets\n\nReturn the categorty label only (e.g., 5), no extra words.\"\"\""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "36 Hours in Lake Tahoe; Straddling Nevada and California, Lake Tahoe is a winter playground for skiers and snowboarders, food enthusiasts and anyone willing to be pulled into the area's adventurous orbit.\nExpand"
        },
        {
          "type": "input_image",
          "image_url": "https://static01.nyt.com/images/2015/03/01/travel/01HOURS2/01HOURS2-watch308-v2.jpg"
        }
      ]
    },
    {
      "type": "reasoning",
      "id": "rs_68263a6e1e5c8191856e1f5b5ee8c77a0252b23c99ac10a1",
      "summary": []
    },
    {
      "id": "msg_68263a70b4208191bb874acec2ab4e8e0252b23c99ac10a1",
      "role": "assistant",
      "content": [
        {
          "type": "output_text",
          "text": "4"
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={
    "effort": "medium"
  },
  tools=[],
  store=True
)



In [12]:
price_estimation (response_5_reasoning,
                  num_rows = 4000,
                  input_cost_per_million = 1.24,
                  output_cost_per_million = 10,
                  num_votes = 3)

# https://openai.com/api/pricing/


🧮 Estimated Cost for 12,000 calls (4,000 rows × 3 votes)
• Avg prompt tokens/call:     687
• Avg completion tokens/call: 135
• Pricing ($/1M tokens): prompt=$1.24, completion=$10
💰 Total: $26.4226    (±10% → $23.7803–$29.0648)



26.422559999999997

In [6]:
response_5_nano_non_reasoning = client.responses.create(
  model="gpt-5-nano",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": "\"\"\"You are given a short news article. Based on its content, return the most appropriate category label from the list below. Respond with only a number from 1 to 24, corresponding to the best-matching category:\n\n1. Health – medical news, public health, fitness, mental health, wellness\n2. Science – scientific discoveries, research studies, space, innovations\n3. Television – TV shows, reviews, industry news, streaming\n4. Travel – tourism, destinations, travel guides, airlines, vacation trends\n5. Movies – film news, reviews, box office, upcoming releases\n6. Dance – ballet, contemporary styles, street dance, performances\n7. Real Estate – housing trends, property sales, architecture, urban planning\n8. Economy – macroeconomics, inflation, markets, GDP, financial policies\n9. Sports – competitions, athletes, Demo_results, professional sports\n10. Theater – plays, Broadway, live performances, stage reviews\n11. Opinion – editorials, commentary, analysis\n12. Music – albums, artists, concerts, festivals, music trends\n13. Books – literature, bestsellers, author interviews, book reviews\n14. Art & Design – fine arts, museums, exhibitions, visual/design trends\n15. Style – fashion trends, beauty, personal style, aesthetics\n16. Media – journalism, publishing, digital media, mass communication\n17. Food – restaurants, recipes, cooking, culinary culture\n18. Well – lifestyle, self-care, mental well-being, personal development\n19. Fashion – clothing, designers, fashion weeks, industry insights\n20. Technology – AI, gadgets, software, cybersecurity, tech innovations\n21. Your Money – personal finance, investing, budgeting, financial planning\n22. Education – schools, universities, learning methods, education policies\n23. Automobiles – car news, EVs, reviews, industry trends\n24. Global Business – international trade, corporations, mergers, global markets\n\nReturn the categorty label only (e.g., 5), no extra words.\"\"\""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "36 Hours in Lake Tahoe; Straddling Nevada and California, Lake Tahoe is a winter playground for skiers and snowboarders, food enthusiasts and anyone willing to be pulled into the area's adventurous orbit.\nExpand"
        },
        {
          "type": "input_image",
          "image_url": "https://static01.nyt.com/images/2015/03/01/travel/01HOURS2/01HOURS2-watch308-v2.jpg"
        }
      ]
    },
    {
      "type": "reasoning",
      "id": "rs_68263a6e1e5c8191856e1f5b5ee8c77a0252b23c99ac10a1",
      "summary": []
    },
    {
      "id": "msg_68263a70b4208191bb874acec2ab4e8e0252b23c99ac10a1",
      "role": "assistant",
      "content": [
        {
          "type": "output_text",
          "text": "4"
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={
    "effort": "minimal"
  },
  tools=[],
  store=True
)



In [7]:
response_5_nano_reasoning = client.responses.create(
  model="gpt-5-nano",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": "\"\"\"You are given a short news article. Based on its content, return the most appropriate category label from the list below. Respond with only a number from 1 to 24, corresponding to the best-matching category:\n\n1. Health – medical news, public health, fitness, mental health, wellness\n2. Science – scientific discoveries, research studies, space, innovations\n3. Television – TV shows, reviews, industry news, streaming\n4. Travel – tourism, destinations, travel guides, airlines, vacation trends\n5. Movies – film news, reviews, box office, upcoming releases\n6. Dance – ballet, contemporary styles, street dance, performances\n7. Real Estate – housing trends, property sales, architecture, urban planning\n8. Economy – macroeconomics, inflation, markets, GDP, financial policies\n9. Sports – competitions, athletes, Demo_results, professional sports\n10. Theater – plays, Broadway, live performances, stage reviews\n11. Opinion – editorials, commentary, analysis\n12. Music – albums, artists, concerts, festivals, music trends\n13. Books – literature, bestsellers, author interviews, book reviews\n14. Art & Design – fine arts, museums, exhibitions, visual/design trends\n15. Style – fashion trends, beauty, personal style, aesthetics\n16. Media – journalism, publishing, digital media, mass communication\n17. Food – restaurants, recipes, cooking, culinary culture\n18. Well – lifestyle, self-care, mental well-being, personal development\n19. Fashion – clothing, designers, fashion weeks, industry insights\n20. Technology – AI, gadgets, software, cybersecurity, tech innovations\n21. Your Money – personal finance, investing, budgeting, financial planning\n22. Education – schools, universities, learning methods, education policies\n23. Automobiles – car news, EVs, reviews, industry trends\n24. Global Business – international trade, corporations, mergers, global markets\n\nReturn the categorty label only (e.g., 5), no extra words.\"\"\""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "36 Hours in Lake Tahoe; Straddling Nevada and California, Lake Tahoe is a winter playground for skiers and snowboarders, food enthusiasts and anyone willing to be pulled into the area's adventurous orbit.\nExpand"
        },
        {
          "type": "input_image",
          "image_url": "https://static01.nyt.com/images/2015/03/01/travel/01HOURS2/01HOURS2-watch308-v2.jpg"
        }
      ]
    },
    {
      "type": "reasoning",
      "id": "rs_68263a6e1e5c8191856e1f5b5ee8c77a0252b23c99ac10a1",
      "summary": []
    },
    {
      "id": "msg_68263a70b4208191bb874acec2ab4e8e0252b23c99ac10a1",
      "role": "assistant",
      "content": [
        {
          "type": "output_text",
          "text": "4"
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={
    "effort": "medium"
  },
  tools=[],
  store=True
)



In [8]:
price_estimation (response_5_nano_reasoning,
                  num_rows = 4000,
                  input_cost_per_million = 0.05,
                  output_cost_per_million = 0.4,
                  num_votes = 3)

# https://openai.com/api/pricing/


🧮 Estimated Cost for 12,000 calls (4,000 rows × 3 votes)
• Avg prompt tokens/call:     643
• Avg completion tokens/call: 71
• Pricing ($/1M tokens): prompt=$0.05, completion=$0.4
💰 Total: $0.7266    (±10% → $0.6539–$0.7993)



0.7266000000000001

In [9]:
price_estimation (response_5_nano_non_reasoning,
                  num_rows = 4000,
                  input_cost_per_million = 0.05,
                  output_cost_per_million = 0.4,
                  num_votes = 3)

# https://openai.com/api/pricing/


🧮 Estimated Cost for 12,000 calls (4,000 rows × 3 votes)
• Avg prompt tokens/call:     643
• Avg completion tokens/call: 7
• Pricing ($/1M tokens): prompt=$0.05, completion=$0.4
💰 Total: $0.4194    (±10% → $0.3775–$0.4613)



0.41940000000000005

## Test with whole sample

In [13]:
GPT_41 = classification_GPT(
    text_path = "Data_test/nytimes_test_sample4000.csv",
    image_dir = "Data_test/imgs_test_sample4000",
    category = category,
    prompt = prompt_GPT,
    column_4_labeling=["headline", "abstract"],
    model = "gpt-4.1",
    api_key = api_key,
    temperature= 1, #a value range from 0 to 2
    mode = "both",
    output_column_name="GPT_41",
    num_themes = 1,
    num_votes = 3,
    wait_time = 0)

2025-10-08 16:13:50.729 | INFO     | labelgenius:print_env_info:101 - [ENV] Python: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
2025-10-08 16:13:50.730 | INFO     | labelgenius:print_env_info:102 - [ENV] OpenAI SDK: 1.70.0
2025-10-08 16:13:50.730 | INFO     | labelgenius:print_env_info:103 - [ENV] Platform: macOS-15.6-arm64-arm-64bit
Classifying final_class: 100%|██████████| 4000/4000 [2:02:31<00:00,  1.84s/item]  


In [14]:
GPT_41.to_excel("Result/GPT_41.xlsx", index=False)

In [5]:
GPT_5_nr = classification_GPT(
    text_path = "Data_test/nytimes_test_sample4000.csv",
    image_dir = "Data_test/imgs_test_sample4000",
    category = category,
    prompt = prompt_GPT,
    column_4_labeling=["headline", "abstract"],
    model = "gpt-5",
    api_key = api_key,
    reasoning_effort= "minimal", #minial for non reasoning
    mode = "both",
    output_column_name="GPT_5_nr",
    num_themes = 1,
    num_votes = 3,
    wait_time = 0.1)

2025-10-08 23:34:58.611 | INFO     | labelgenius.LabelGenius:print_env_info:101 - [ENV] Python: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
2025-10-08 23:34:58.612 | INFO     | labelgenius.LabelGenius:print_env_info:102 - [ENV] OpenAI SDK: 1.70.0
2025-10-08 23:34:58.612 | INFO     | labelgenius.LabelGenius:print_env_info:103 - [ENV] Platform: macOS-15.6-arm64-arm-64bit
Classifying final_class: 100%|██████████| 4000/4000 [2:44:10<00:00,  2.46s/item]  


In [6]:
GPT_5_nr.to_excel("Result/GPT_5_nr.xlsx", index=False)

In [5]:
GPT_5_r = classification_GPT(
    text_path = "Data_test/nytimes_test_sample4000.csv",
    image_dir = "Data_test/imgs_test_sample4000",
    category = category,
    prompt = prompt_GPT,
    column_4_labeling=["headline", "abstract"],
    model = "gpt-5",
    api_key = api_key,
    reasoning_effort= "medium", #low, medium, hih
    mode = "both",
    output_column_name="GPT_5_r",
    num_themes = 1,
    num_votes = 3,
    wait_time = 0.1)

2025-10-13 18:57:32.341 | INFO     | labelgenius.LabelGenius:print_env_info:101 - [ENV] Python: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
2025-10-13 18:57:32.342 | INFO     | labelgenius.LabelGenius:print_env_info:102 - [ENV] OpenAI SDK: 1.70.0
2025-10-13 18:57:32.342 | INFO     | labelgenius.LabelGenius:print_env_info:103 - [ENV] Platform: macOS-15.6-arm64-arm-64bit
Classifying final_class: 100%|██████████| 4000/4000 [6:13:01<00:00,  5.60s/item]  


In [6]:
GPT_5_r.to_excel("Result/GPT_5_r.xlsx", index=False)

In [7]:
GPT_5_nano_r = classification_GPT(
    text_path = "Data_test/nytimes_test_sample4000.csv",
    image_dir = "Data_test/imgs_test_sample4000",
    category = category,
    prompt = prompt_GPT,
    column_4_labeling=["headline", "abstract"],
    model = "gpt-5-nano",
    api_key = api_key,
    reasoning_effort= "medium", #low, medium, hih
    mode = "both",
    output_column_name="GPT_5_nano_r",
    num_themes = 1,
    num_votes = 3,
    wait_time = 0.1)

2025-10-14 01:10:56.556 | INFO     | labelgenius.LabelGenius:print_env_info:101 - [ENV] Python: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
2025-10-14 01:10:56.558 | INFO     | labelgenius.LabelGenius:print_env_info:102 - [ENV] OpenAI SDK: 1.70.0
2025-10-14 01:10:56.559 | INFO     | labelgenius.LabelGenius:print_env_info:103 - [ENV] Platform: macOS-15.6-arm64-arm-64bit
Classifying final_class: 100%|██████████| 4000/4000 [4:03:16<00:00,  3.65s/item]  


In [9]:
GPT_5_nano_r.to_excel("Result/GPT_5_nano_r.xlsx", index=False)

In [10]:
GPT_5_nano_nr = classification_GPT(
    text_path = "Data_test/nytimes_test_sample4000.csv",
    image_dir = "Data_test/imgs_test_sample4000",
    category = category,
    prompt = prompt_GPT,
    column_4_labeling=["headline", "abstract"],
    model = "gpt-5-nano",
    api_key = api_key,
    reasoning_effort= "minimal", #low, medium, hih
    mode = "both",
    output_column_name="GPT_5_nano_nr",
    num_themes = 1,
    num_votes = 3,
    wait_time = 0.1)

2025-10-14 15:27:29.318 | INFO     | labelgenius.LabelGenius:print_env_info:101 - [ENV] Python: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
2025-10-14 15:27:29.321 | INFO     | labelgenius.LabelGenius:print_env_info:102 - [ENV] OpenAI SDK: 1.70.0
2025-10-14 15:27:29.322 | INFO     | labelgenius.LabelGenius:print_env_info:103 - [ENV] Platform: macOS-15.6-arm64-arm-64bit
Classifying final_class: 100%|██████████| 4000/4000 [5:27:58<00:00,  4.92s/item]     


In [11]:
GPT_5_nano_nr.to_excel("Result/GPT_5_nano_nr.xlsx", index=False)